In [1]:
import os
os.environ["NVIDIA_API_KEY"] = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"

In [40]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
llm_nvidia = ChatNVIDIA(model="mistralai/mixtral-8x7b-instruct-v0.1")

In [41]:
from langchain_community.chat_models import ChatOllama

llm_ollama = ChatOllama(model="llama3")

In [42]:
llm = llm_nvidia

# Agents

In [43]:
#!pip install langchain-experimental
# !pip install numexpr
#!pip install wikipedia
#!pip install langchainhub

In [44]:
from langchain.agents import AgentExecutor, load_tools, create_react_agent 
from langchain_core.tools import Tool
from langchain_experimental.utilities import PythonREPL
from langchain.prompts import PromptTemplate

In [45]:
tools = load_tools(["llm-math","wikipedia"], llm=llm)

In [46]:
prompt_text = '''Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}'''
prompt = PromptTemplate.from_template(prompt_text)
agent = create_react_agent(llm = llm_nvidia, tools = tools, prompt = prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [47]:
question = {
    "input": "What is the 25% of 300?"
}

result = agent_executor.invoke(question)
print(result['output'])

The 25% of 300 is 75.0.


## Wikipedia example

In [48]:
llm = ChatNVIDIA(model="mistralai/mixtral-8x7b-instruct-v0.1", temperature=0.0)

In [49]:
question = {
    "input" : "Tom Michael Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU)\
what book did he write?"
}
result = agent_executor.invoke(question) 
print(result['output'])

Tom Michael Mitchell is a computer scientist and the Founders University Professor at Carnegie Mellon University. He is the author of the book "Machine Learning".


## Python Agent

In [50]:
python_repl = PythonREPL()
python_repl.run("print(2+2)")

Python REPL can execute arbitrary code. Use with caution.


'4\n'

In [51]:
repl_tool = Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
    func=python_repl.run,
)
agent = create_react_agent(llm = llm, tools = [repl_tool], prompt = prompt)
agent_executor = AgentExecutor(agent=agent, tools=[repl_tool], handle_parsing_errors=True)


In [52]:
customer_list = [["Harrison", "Chase"], 
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"], 
                 ["Geoff","Fusion"], 
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]

In [53]:
question = {
    "input": f"""Sort these customers by last name 
    and then first name 
    and print the output: {customer_list}"""
}

result = agent_executor.invoke(question)

In [54]:
print(result['output'])

```python
Customers = [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]
sorted_customers = sorted(Customers, key=lambda x: (x[0], x[1]))
for customer in sorted_customers:
    print(customer)
```
